# ASTR-113: Real Data Loading Smoke Test

This notebook validates the real-data collection and loading pipeline:
- Collect validated detections into a `TrainingDataset`
- Persist `TrainingSample` rows
- Load the dataset and create train/val/test splits

Run this before integrating with `notebooks/training/model_training.ipynb`. 


In [1]:
# Setup imports and environment
import os, sys
from pathlib import Path

project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root))

print(f"Project root added: {project_root}")


Project root added: /home/chris/github/AstrID


In [2]:
API_BASE = os.environ.get("ASTRID_API_BASE", "http://127.0.0.1:8000")
API_BASE


'http://127.0.0.1:8000'

In [3]:
from src.core.constants import TRAINING_PIPELINE_API_KEY

global AUTH_HEADERS
AUTH_HEADERS = {
    "X-API-Key": TRAINING_PIPELINE_API_KEY,
    "Content-Type": "application/json",
}

In [4]:
# Collect a small dataset via API
import requests
from datetime import datetime

params = {
    "survey_ids": ["hst"],
    "start": "2024-01-01T00:00:00",
    "end": "2024-12-31T23:59:59",
    "confidence_threshold": 0.7,
    "max_samples": 50,
    "name": "smoketest_hst_2024"
}

r = requests.post(f"{API_BASE}/training/datasets/collect", json=params, headers=AUTH_HEADERS, timeout=60)
r.raise_for_status()
resp = r.json()
resp


{'status': 'success',
 'data': {'dataset_id': 'f6e72a79-0fbc-4445-8dad-e10e3d959bd9',
  'name': 'smoketest_hst_2024',
  'total': 0,
  'quality': {'anomaly_ratio': 0.0, 'quality_score': 0.0}},
 'meta': {},
 'error': None}

In [5]:
# Verify dataset is listed
r = requests.get(f"{API_BASE}/training/datasets", headers=AUTH_HEADERS)
r.raise_for_status()
datasets = r.json()["data"]

print(f"Datasets: {len(datasets)}")
# Show the most recent one
sorted(datasets, key=lambda d: d.get("created_at", ""), reverse=True)[:3]


Datasets: 12


[{'id': 'f6e72a79-0fbc-4445-8dad-e10e3d959bd9',
  'name': 'smoketest_hst_2024',
  'total_samples': 0,
  'quality_score': 0.0,
  'status': 'active',
  'created_at': '2025-09-23T06:45:17.964315+00:00'},
 {'id': '0acda9eb-a45d-448a-a434-ae1806f2bb99',
  'name': 'test_85a67de7-709e-49d8-a04c-4e4f07c548a2_20250922_234403',
  'total_samples': 0,
  'quality_score': 0.0,
  'status': 'active',
  'created_at': '2025-09-23T06:44:04.662205+00:00'},
 {'id': '7d26d30a-3af5-4597-b5a2-3bbb5c3f1b8d',
  'name': 'test_dad7665f-907b-4bcd-bf77-237501714edf_20250922_220709',
  'total_samples': 0,
  'quality_score': 0.0,
  'status': 'active',
  'created_at': '2025-09-23T05:07:09.989516+00:00'}]

In [6]:
# Inspect dataset via API (no direct DB access needed in notebook)
# Resolve dataset_id from prior response or fallback to latest dataset via API
if isinstance(resp, dict) and "data" in resp and resp["data"].get("dataset_id"):
    dataset_id = resp["data"]["dataset_id"]
else:
    r = requests.get(f"{API_BASE}/training/datasets", headers=AUTH_HEADERS)
    r.raise_for_status()
    datasets = r.json().get("data", [])
    if not datasets:
        raise RuntimeError("No training datasets available")
    # pick the most recent
    datasets_sorted = sorted(datasets, key=lambda d: d.get("created_at", ""), reverse=True)
    dataset_id = str(datasets_sorted[0]["id"])  # ensure string

print("Dataset ID:", dataset_id)

# Get dataset details
detail = requests.get(f"{API_BASE}/training/datasets/{dataset_id}", headers=AUTH_HEADERS)
detail.raise_for_status()
detail.json()


Dataset ID: f6e72a79-0fbc-4445-8dad-e10e3d959bd9


{'status': 'success',
 'data': {'id': 'f6e72a79-0fbc-4445-8dad-e10e3d959bd9',
  'name': 'smoketest_hst_2024',
  'total_samples': 0,
  'quality_score': 0.0,
  'status': 'active',
  'samples': 0},
 'meta': {},
 'error': None}

In [7]:
# Optional: preview a few sample image paths from DB via API
# (Future enhancement: API could return sample listings)
print("For now, verify counts above. Integration with training notebook will consume by dataset_id.")


For now, verify counts above. Integration with training notebook will consume by dataset_id.
